# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:
import numpy as np
import torch
from torchvision import transforms
import torchvision
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from os.path import isfile, join
import PIL
from IPython.display import clear_output
import os
from PIL import Image
from sklearn.metrics import accuracy_score
from typing import Dict, List, Tuple, Union
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
# You may add any imports you need

In [2]:
!wget https://www.dropbox.com/s/33l8lp62rmvtx40/dataset.zip
!unzip '/content/dataset.zip'
!rm '/content/dataset.zip'



Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: dataset/dataset/val/class_101/00048.jpg  
  inflating: dataset/dataset/val/class_101/00049.jpg  
   creating: dataset/dataset/val/class_102/
  inflating: dataset/dataset/val/class_102/00000.jpg  
  inflating: dataset/dataset/val/class_102/00001.jpg  
  inflating: dataset/dataset/val/class_102/00002.jpg  
  inflating: dataset/dataset/val/class_102/00003.jpg  
  inflating: dataset/dataset/val/class_102/00004.jpg  
  inflating: dataset/dataset/val/class_102/00005.jpg  
  inflating: dataset/dataset/val/class_102/00006.jpg  
  inflating: dataset/dataset/val/class_102/00007.jpg  
  inflating: dataset/dataset/val/class_102/00008.jpg  
  inflating: dataset/dataset/val/class_102/00009.jpg  
  inflating: dataset/dataset/val/class_102/00010.jpg  
  inflating: dataset/dataset/val/class_102/00011.jpg  
  inflating: dataset/dataset/val/class_102/00012.jpg  
  inflating: dataset/dataset/val/class_102/00013.jpg  
  inflat

In [3]:
img = Image.open('/content/dataset/dataset/train/class_000/00001.jpg')
print(f"Image format: {img.format}; shape: {img.size}; color scheme: {img.mode}")

Image format: JPEG; shape: (64, 64); color scheme: RGB


In [23]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [22]:
import numpy as np
import random
set_random_seed(42)

### Подготовка данных

In [24]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Normalize, Resize, ToTensor 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# transforms.Compose([
# >>>     transforms.CenterCrop(10),
# >>>     transforms.PILToTensor(),
# >>>     transforms.ConvertImageDtype(torch.float),
# >>> ])

train_transform = transforms.Compose(
    [
        transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)
val_transform = transforms.Compose(
    [
        transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)


train_dataset = ImageFolder(
    "/content/dataset/dataset/train/", 
    transform=train_transform

)


val_dataset = ImageFolder(
    "/content/dataset/dataset/val/", 
    transform= val_transform

)

# YOU CAN DEFINE AUGMENTATIONS HERE

# REPLACE ./dataset/dataset WITH THE FOLDER WHERE YOU DOWNLOADED AND UNZIPPED THE DATASET
# OR USE torchvision.datasets.ImageFolder INSTEAD OF MyDataset

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=64,
    shuffle=True, 
    num_workers=1)
    
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, 
    batch_size=64,
    shuffle=False, 
    num_workers=1)

In [21]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [32]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, epoch, device="cuda:0"):
    model.train()
    for imgs, labels in tqdm(train_dataloader, desc=f"Training, epoch {epoch}", leave=False):
        train_epoch_loss, train_epoch_true_hits = torch.empty(0).to(device), torch.empty(0).to(device)
        imgs, labels = imgs.to(device), labels.to(device)
        torch.cuda.synchronize()
        y_pred = model(imgs)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        # log loss for the current epoch and the whole training history
        train_epoch_loss = torch.cat((train_epoch_loss, loss.unsqueeze(0) / labels.size(0)))

        # log accuracy for the current epoch and the whole training history
        pred_classes = torch.argmax(y_pred, dim=-1)
        train_epoch_true_hits = torch.cat((
            train_epoch_true_hits, 
            (pred_classes == labels).sum().unsqueeze(0)
        ))

    return model, \
    (loss.data / labels.size(0)), \
    ((pred_classes == labels).cpu().sum() / labels.shape[0]), \
    train_epoch_loss,\
    train_epoch_true_hits



def predict(model, val_dataloder, criterion, device="cuda:0"):
      val_epoch_loss, val_epoch_true_hits, predicted_classes = torch.empty(0).to(device), torch.empty(0).to(device), torch.empty(0).to(device)
      true_classes = torch.empty(0).to(device)
      
      model.eval()
      with torch.no_grad():
          for imgs, labels in tqdm(val_dataloader, leave=False):
                imgs, labels = imgs.to(device), labels.to(device)

                y_pred = model(imgs)
                loss = criterion(y_pred, labels)
                val_epoch_loss = torch.cat((val_epoch_loss, loss.unsqueeze(0) / labels.size(0)))

                pred_classes = torch.argmax(y_pred, dim=-1)
                val_epoch_true_hits = torch.cat((
                    val_epoch_true_hits,
                    (pred_classes == labels).sum().unsqueeze(0)))


                true_classes = torch.cat((true_classes, labels))
                predicted_classes = torch.cat((predicted_classes, pred_classes))

      losses = val_epoch_loss.mean().item()
      predicted_classes = predicted_classes.cpu()
      true_classes = true_classes.cpu()

      return losses, predicted_classes, true_classes


def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=1, scheduler=None):
    model.to(device)
            # Train, evaluate, print accuracy, make a step of scheduler or whatever you want...
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    for epoch in range(n_epochs):
        model, train_loss, train_acc, train_epoch_loss, train_epoch_true_hits  = train_one_epoch(model, 
                                                                                                 train_dataloader, 
                                                                                                 criterion, 
                                                                                                 optimizer, 
                                                                                                 epoch,
                                                                                                 device="cuda:0") 
        if (epoch % 2 == 0) & (epoch != 0):
          scheduler.step()
                                                                                                 
                                                                                                  
                                                                                                  
        train_loss_log.append(train_loss)
        train_acc_log.append(train_acc)

        # валидация
        val_epoch_loss, val_epoch_true_hits = torch.empty(0).to(device), torch.empty(0).to(device)
        model.eval()
        with torch.no_grad():
            for imgs, labels in tqdm(val_dataloader, desc=f"Validating, epoch {epoch}", leave=False):
                imgs, labels = imgs.to(device), labels.to(device)

                y_pred = model(imgs)
                loss = criterion(y_pred, labels)
                val_epoch_loss = torch.cat((val_epoch_loss, loss.unsqueeze(0) / labels.size(0)))

                pred_classes = torch.argmax(y_pred, dim=-1)
                val_epoch_true_hits = torch.cat((
                    val_epoch_true_hits,
                    (pred_classes == labels).sum().unsqueeze(0)
                ))

        val_loss_log.append(val_epoch_loss.mean())
        val_acc_log.append(val_epoch_true_hits.sum() / val_epoch_true_hits.size(0) / val_dataloader.batch_size)
        clear_output()

        print("Train loss:", train_epoch_loss.mean().item())
        print(
            "Train acc:", 
            (train_epoch_true_hits.sum() / train_epoch_true_hits.size(0) / train_dataloader.batch_size).item()
        )
        print("Val loss:", val_epoch_loss.mean().item())
        print(
            "Val acc:", 
            (val_epoch_true_hits.sum() / val_epoch_true_hits.size(0) / val_dataloader.batch_size).item()
        )



### Обучение модели, запуски экспериментов

In [25]:
from torch import optim
import torchvision.models as models

device = 'cuda:0'

model = models.resnet18(pretrained=False) # МОДЕЛЬ - RESNET
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# эта модель дает скор 39, но на 20 эпохах...... Времени не было уже ее запускать

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.out1 = nn.Sequential(
        nn.Conv2d(3, 64, 3, padding='same'),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, 3, padding='same'),
        nn.BatchNorm2d(64),
        nn.PReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.2))

    self.out2 = nn.Sequential(
        nn.Conv2d(64, 32, 3, padding='same'), 
        nn.BatchNorm2d(32),
        nn.PReLU(),
        nn.Conv2d(32, 32, 3, padding='same'),
        nn.BatchNorm2d(32),
        nn.PReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.2))

    self.out3 = nn.Sequential(nn.Conv2d(
        32, 64, 3, padding='same'),
        nn.BatchNorm2d(64),
        nn.PReLU(),
        nn.Conv2d(64, 128, 3, padding='same'),
        nn.BatchNorm2d(128),
        nn.PReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.2))

    self.out4 = nn.Sequential(nn.Flatten(),
        nn.Linear(64 * 32 * 4, 128),
        nn.BatchNorm1d(128),
        nn.PReLU(),
        nn.Dropout(0.4),
        nn.Linear(128, 200),
        nn.LogSoftmax())

  def forward(self, x):
      x1 = self.out1(x)
      x2 = self.out2(x1)
      x3 = self.out3(x2)
      x4 = self.out4(x3)
      return x4


Простой тест на проверку правильности написанного кода

In [26]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

tests passed


Запустить обучение можно в ячейке ниже.

In [28]:
train(model, 
      train_dataloader, 
      val_dataloader, 
      criterion, 
      optimizer, 
      scheduler=scheduler, 
      device="cuda:0") # тут 12 эпох

Train loss: 0.05102395638823509
Train acc: 0.25
Val loss: 0.04420442134141922
Val acc: 0.38385748863220215


In [35]:
train(model, 
      train_dataloader, 
      val_dataloader, 
      criterion, 
      optimizer, 
      scheduler=scheduler, 
      device="cuda:0") #здесь 13-ая доп и 14-ая доп эпохи, но на 14 произошло переобучение, поэтому оставляю за 13-ую.

Train loss: 0.04521898925304413
Train acc: 0.3125
Val loss: 0.047418929636478424
Val acc: 0.37649282813072205


### Проверка полученной accuracy

После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, реализуйте и запустите функцию `evaluate`. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

# НИЖЕ ПРЕДСТАВЛЕН СКОР ЗА 13 ЭПОХ

In [34]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за это задание составит {} баллов".format(min(10, 10 * accuracy / 0.44))) #скор за 13 эпох

Оценка за это задание составит 8.702272727272728 баллов


### Отчёт об экспериментах 

текст писать тут

In [ ]:
# использовала материалы семов

In [ ]:
# resnet дает 13 на нулевой эпохе, но быстро скисает на четвертой эпохе
# vgg - плохо
# alexanet - плохо
# mnasnet1_0 - плохо
# efficientnet_b0 - плохо

In [ ]:
# Итоговая модель - resnet, все ниже представленное - эксперименты с собственной моделью

In [ ]:
# тут все отчеты с кодом
https://mountainous-crowberry-5fe.notion.site/HW_2-106076c5de2848fa95fd6b347f9d7415

In [ ]:
train(model, 
      train_dataloader, 
      val_dataloader, 
      criterion, 
      optimizer, 
      scheduler=scheduler, 
      device="cuda:0") # c sheduler # 4 эпоха 19 #моменум 0.4, шаг 0.01  на второй эпохе 17! 22.7

Train loss: 0.22064687311649323
Train acc: 0.3125
Val loss: 0.21410883963108063
Val acc: 0.22769999504089355


In [ ]:
train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0") #без sheduler 5 эпохи (на 5-ой acc 22)

Train loss: 0.2760438621044159
Train acc: 0.25
Val loss: 0.21791960299015045
Val acc: 0.22019998729228973


In [ ]:
train(model, 
      train_dataloader, 
      val_dataloader, # batch 16
      criterion, 
      optimizer, 
      scheduler=scheduler, 
      device="cuda:0") # c sheduler шаг 0.01  на аятой эпохе 23

Train loss: 0.22873038053512573
Train acc: 0.0
Val loss: 0.20482346415519714
Val acc: 0.2563000023365021


In [ ]:
train(model, 
      train_dataloader, 
      val_dataloader, # batch 16
      criterion, 
      optimizer, 
      scheduler=scheduler, 
      device="cuda:0") # c sheduler шаг 0.01  на аятой эпохе 23

Train loss: 0.20643173158168793
Train acc: 0.25
Val loss: 0.20649360120296478
Val acc: 0.2523999810218811


In [ ]:
train(model, 
      train_dataloader, 
      val_dataloader, 
      criterion, 
      optimizer, 
      scheduler=scheduler, 
      device="cuda:0") # c sheduler # 4 эпоха 19

Train loss: 0.2244097739458084
Train acc: 0.1875
Val loss: 0.2261524349451065
Val acc: 0.19849999248981476


KeyboardInterrupt: ignored

In [ ]:
train(model, 
      train_dataloader, 
      val_dataloader, # batch 16
      criterion, 
      optimizer, 
      scheduler=scheduler, 
      device="cuda:0") # c sheduler шаг 0.01  на аятой эпохе 23

Train loss: 0.2696867883205414
Train acc: 0.0625
Val loss: 0.20985080301761627
Val acc: 0.23629999160766602
